In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve

# Read Data

In [ ]:
# TODO: read data.csv into a dataframe (~1 line)

## END OF TODO

df.head()

In [ ]:
df.shape
# Expected output: (1040,29)

In [ ]:
del df['id'] # remove subject ID column

In [ ]:
df.describe()

# Split Data into Training and Testing

In [ ]:
# TODO: extract features and labels (~3 lines)
# Use the last column as label Y. 
# The next to the last column UPDRS (Unified Parkinson's Disease Rating Scale)
# can be used in the bonus (optional) problem for regression. 
# Use the rest columns (other than the last two columns) as features X


# END OF TODO 

In [ ]:
# TODO: hold out 20% for testing using stratified sampling (~1 line)

# END OF TODO 

In [ ]:
nfolds = 10

In [ ]:
X.shape # Expected outcome: (1040,26)

In [ ]:
Y.shape # Expected outcome: (1040,0)

## K-Nearest Neighbors (KNN)

In [ ]:
pipe_knn = Pipeline([('scl', StandardScaler()),
                     ('knn', KNeighborsClassifier())])

# TODO: set parameter k as odd numbers from 1 to 13 (~1 line)

# END OF TODO 

param_grid = [{'knn__n_neighbors': param_range}]

# TODO: Using GridSearchCV to find the optimal k. setup n_jobs=-1 to use all CPUs (~1 line)


# TODO: Fit the KNN grid search model to training data (~1 line)

# END OF TODO

print(gs_knn.best_score_)
print(gs_knn.best_params_)

In [ ]:
clf_knn = gs_knn.best_estimator_
clf_knn.fit(X_train, Y_train)
print('Test accuracy: %.3f' % clf_knn.score(X_test, Y_test))

## Support Vector Machines (SVM)

In [ ]:
# TODO: Build a pipeline including a standard scaler and SVC classifier (~1 line)

# END OF TODO

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# TODO: Use GridSearchCV to find optimal kernel (linear or rbf), C (param_range), and/or gamma (param_range) (~3 lines)





# END OF TODO

print(gs_svm.best_score_)
print(gs_svm.best_params_)

In [ ]:
clf_svm = gs_svm.best_estimator_
clf_svm.fit(X_train, Y_train)
print('Test accuracy: %.3f' % clf_svm.score(X_test, Y_test))

## Naive Bayes (NB)

In [ ]:
# TODO: Build a Naive Bayes classifier (no hyper-parameters) (~2 lines)


# END OF TODO

In [ ]:
print(clf_nb.score(X_test,Y_test))

## Neural Networks (NN)

In [ ]:
# TODO: Use MLPClassifier to build a neural network.(~2 lines)
# Try different hyper-parameters of # of layers and # of nodes in each layer



# END OF TODO

In [ ]:
print(clf_nn.score(X_test,Y_test))

## Decision Tree

In [ ]:
## TODO: Use tree.DecisionTreeClassifier to build a decision tree using training data (~2 lines)


## END OF TODO
print(clf_dt.score(X_test,Y_test))

## Visualize the tree

In [ ]:
import graphviz as gv
dot_data = tree.export_graphviz(clf_dt, out_file=None) 
graph = gv.Source(dot_data) 
graph.render("Parkinson Decision Tree")
feature_names = list(df.iloc[:, 0:-2].head())

In [ ]:
dot_data = tree.export_graphviz(clf_dt, out_file=None,
                                feature_names=feature_names,
                                class_names=['HC','PD'],
                                filled=True,
                                rounded=True)  
graph = gv.Source(dot_data)
graph

# Learning Curve

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
title = "Learning Curves (KNN)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=1)
estimator = gs_knn.best_estimator_
plot_learning_curve(estimator, title, X_train, Y_train, ylim=(0.4, 1.02), cv=cv, n_jobs=-1)
plt.show()

In [ ]:
## TODO: Plot the learning and cross-validaiton curves for SVM (~4 lines)




# END OF TODO

In [ ]:
## TODO: Plot the learning and cross-validaiton curves for Naive Bayes (~4 lines)




# END OF TODO

In [ ]:
## TODO: Plot the learning and cross-validaiton curves for Neural Networks (~4 lines)




# END OF TODO

In [ ]:
## TODO: Plot the learning and cross-validaiton curves for Decision Tree (~4 lines)




# END OF TODO

# BONUS (OPTIONAL): Regression

In [ ]:
# TODO: Use XGBoost Regression or Support Vector Regression to train a regression model to predict UPDRS (bonus: 5 points)



# END OF TODO